In [1]:
from pathlib import Path
import pandas as pd

PATH_NOIPS = Path(r"D:\BOOSTIFY-IOTML\snort\pengujian\webbrute\hardware_log_20251213_234910.csv")
PATH_IPS   = Path(r"D:\BOOSTIFY-IOTML\snort\pengujian_model\ae\webbrute\hardware_log_20251215_023336.csv")

RAM_TOTAL_MB = 8192


In [2]:
def extract_metrics(csv_path):
    df = pd.read_csv(csv_path)
    df["timestamp"] = pd.to_datetime(df["timestamp"])

    df["cpu_used_pct"] = 100 - df["cpu_idle_pct"]
    df["ram_used_mb"]  = RAM_TOTAL_MB - df["ram_available_mb"]
    df["disk_io_kbs"]  = df["disk_read_kbs"] + df["disk_write_kbs"]
    df["net_io_kbs"]   = df["net_rx_kbs"] + df["net_tx_kbs"]

    return {
        "cpu_mean": df["cpu_used_pct"].mean(),
        "cpu_max": df["cpu_used_pct"].max(),
        "ram_mean": df["ram_used_mb"].mean(),
        "ram_max": df["ram_used_mb"].max(),
        "disk_mean": df["disk_io_kbs"].mean(),
        "disk_max": df["disk_io_kbs"].max(),
        "net_mean": df["net_io_kbs"].mean(),
        "net_max": df["net_io_kbs"].max(),
    }


In [3]:
def process_single_csv(csv_path, scenario_name, attack_name):
    metrics = extract_metrics(csv_path)
    metrics["scenario"] = scenario_name
    metrics["attack"] = attack_name
    return pd.DataFrame([metrics])


In [4]:
noips_df = process_single_csv(
    PATH_NOIPS,
    scenario_name="no_ips",
    attack_name="goldeneye"
)

ips_df = process_single_csv(
    PATH_IPS,
    scenario_name="ips",
    attack_name="goldeneye"
)

all_df = pd.concat([noips_df, ips_df], ignore_index=True)


In [5]:
print(all_df.columns)
print(all_df)


Index(['cpu_mean', 'cpu_max', 'ram_mean', 'ram_max', 'disk_mean', 'disk_max',
       'net_mean', 'net_max', 'scenario', 'attack'],
      dtype='object')
    cpu_mean  cpu_max     ram_mean  ram_max   disk_mean  disk_max  net_mean  \
0   1.258188     2.17  2997.918658  3003.58   41.891745    448.24  2.430872   
1  23.437500    29.30  3265.817056  3283.61  218.293389  22003.78  2.825667   

   net_max scenario     attack  
0    14.56   no_ips  goldeneye  
1    12.37      ips  goldeneye  


In [6]:
mean_cols = ["cpu_mean", "ram_mean", "disk_mean", "net_mean"]

table_mean = (
    all_df
    .set_index(["attack", "scenario"])[mean_cols]
    .round(2)
)

print(table_mean)


                    cpu_mean  ram_mean  disk_mean  net_mean
attack    scenario                                         
goldeneye no_ips        1.26   2997.92      41.89      2.43
          ips          23.44   3265.82     218.29      2.83


In [7]:
table_compare = (
    all_df
    .set_index(["attack", "scenario"])
    .sort_index()
    .round(2)
)

print(table_compare)


                    cpu_mean  cpu_max  ram_mean  ram_max  disk_mean  disk_max  \
attack    scenario                                                              
goldeneye ips          23.44    29.30   3265.82  3283.61     218.29  22003.78   
          no_ips        1.26     2.17   2997.92  3003.58      41.89    448.24   

                    net_mean  net_max  
attack    scenario                     
goldeneye ips           2.83    12.37  
          no_ips        2.43    14.56  


In [8]:
pivot = all_df.set_index("scenario")

delta_abs = (
    pivot.loc["ips", mean_cols] -
    pivot.loc["no_ips", mean_cols]
).to_frame(name="delta")

print(delta_abs.round(2))


                delta
cpu_mean    22.179312
ram_mean   267.898398
disk_mean  176.401644
net_mean     0.394794


In [9]:
delta_pct = (
    (pivot.loc["ips", mean_cols] - pivot.loc["no_ips", mean_cols])
    / pivot.loc["no_ips", mean_cols]
    * 100
).to_frame(name="delta_percent")

print(delta_pct.round(2))


          delta_percent
cpu_mean    1762.798048
ram_mean       8.936146
disk_mean    421.089272
net_mean      16.240843


In [10]:
import pandas as pd

# ===============================
# 1) Tabel perbandingan utama
# ===============================
table_compare = (
    all_df
    .set_index(["attack", "scenario"])
    .sort_index()
    .round(2)
)

table_compare_out = table_compare.reset_index()
table_compare_out["section"] = "table_compare"


# ===============================
# 2) Delta absolut (IPS - No IPS)
# ===============================
pivot = all_df.set_index("scenario")

delta_abs = (
    pivot.loc["ips", mean_cols] -
    pivot.loc["no_ips", mean_cols]
).round(2).to_frame(name="value")

delta_abs_out = delta_abs.reset_index().rename(columns={"index": "metric"})
delta_abs_out["section"] = "delta_absolute"


# ===============================
# 3) Delta persentase (%)
# ===============================
delta_pct = (
    (pivot.loc["ips", mean_cols] - pivot.loc["no_ips", mean_cols])
    / pivot.loc["no_ips", mean_cols]
    * 100
).round(2).to_frame(name="value")

delta_pct_out = delta_pct.reset_index().rename(columns={"index": "metric"})
delta_pct_out["section"] = "delta_percent"


# ===============================
# 4) Gabungkan semua
# ===============================
final_csv = pd.concat(
    [table_compare_out, delta_abs_out, delta_pct_out],
    ignore_index=True
)

# ===============================
# 5) Simpan ke 1 CSV
# ===============================
final_csv.to_csv("comparison_webbrute.csv", index=False)

print("[INFO] comparison_webbrute.csv saved")


[INFO] comparison_webbrute.csv saved
